In [ ]:
import json

In [ ]:
!pip install emoji
!pip install openai==0.28.1
!pip uninstall typing_extensions
!pip install typing_extensions

In [ ]:
def embedding_word(text, target_word):

  model_checkpoint = "google-bert/bert-base-multilingual-cased"

  # Load pre-trained BERT model and tokenizer
  tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
  model = BertForSequenceClassification.from_pretrained(model_checkpoint)

  # Tokenize the input text and include special tokens ([CLS], [SEP])
  encoded_input = tokenizer(text, return_tensors='pt')

  token_ids = encoded_input['input_ids'][0].tolist()  # Extract token IDs as a list
  target_word_tokens = tokenizer.encode(target_word, add_special_tokens=False)

  # Search for the sequence of token IDs corresponding to the target word
  start_index = next((i for i in range(len(token_ids)) if token_ids[i:i+len(target_word_tokens)] == target_word_tokens), None)
  if start_index is None:
      raise ValueError("Target word not found in the tokenized input.")


  outputs = model(**encoded_input)
  embeddings = outputs[0] #last_hidden_state  # Accessing embeddings directly


  if len(target_word_tokens) > 1:
      word_embedding = embeddings[0, start_index:start_index + len(target_word_tokens)].mean(dim=0)
  else:
      word_embedding = embeddings[0, start_index]


  return word_embedding

def embedding_sent(text):

  # Tokenize the input text and include special tokens ([CLS], [SEP])
  encoded_input = tokenizer(text, return_tensors='pt')

  # Get CLS embeddings
  outputs = model(**encoded_input)
  embeddings = outputs.last_hidden_state  # Tensor of shape (batch_size, sequence_length, hidden_size)

  return embeddings

In [ ]:
embedding_word('انا هو', 'انا')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor(-0.0977, grad_fn=<MeanBackward1>)

In [ ]:
#filtering using camelbert
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertForSequenceClassification
import torch


# Define function to calculate contextual similarity
def get_similarity(question, choice):
    question_emb = embedding_word(question, x)
    choice_emb = embedding_sent(choice)

    similarity_score = cosine_similarity(question_emb, choice_emb)
    return similarity_score

x = 'Hi'

print(get_similarity('hi hello', 'hi is a word'))

In [ ]:
# imports


import pandas as pd
from numpy import dot
from numpy.linalg import norm
import pickle
import openai

from openai.embeddings_utils import (
    get_embedding,
    distances_from_embeddings,
    tsne_components_from_embeddings,
    chart_from_components,
    indices_of_nearest_neighbors_from_distances,
)

# constants
EMBEDDING_MODEL = "text-embedding-ada-002"
openai.api_key = 'add you openai token here'

# establish a cache of embeddings to avoid recomputing
# cache is a dict of tuples (text, model) -> embedding, saved as a pickle file

# set path to embedding cache
embedding_cache_path = "recommendations_embeddings_cache_ruby.pkl"

# load the cache if it exists, and save a copy to disk
try:
    embedding_cache = pd.read_pickle(embedding_cache_path)
except FileNotFoundError:
    embedding_cache = {}
with open(embedding_cache_path, "wb") as embedding_cache_file:
    pickle.dump(embedding_cache, embedding_cache_file)

# define a function to retrieve embeddings from the cache if present, and otherwise request via the API
def embedding_from_string(
    string: str,
    model: str = EMBEDDING_MODEL,
    embedding_cache=embedding_cache
) -> list:
    """Return embedding of given string, using a cache to avoid recomputing."""
    if (string, model) not in embedding_cache.keys():
        embedding_cache[(string, model)] = get_embedding(string, model)
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
    return embedding_cache[(string, model)]

In [ ]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0  # Can't divide by zero, return 0 similarity in this case
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity

In [ ]:
import numpy as np  # Make sure to import numpy

def get_similarity(question, choice):
    question_emb = embedding_from_string(question)
    choice_emb = embedding_from_string(choice)

    similarity_score = cosine_similarity(question_emb, choice_emb)
    return similarity_score

print(get_similarity('Me here', 'Me here'))

0.9999999999999999


In [ ]:
#filtering using camelbert
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Define function to calculate contextual similarity
def get_similarity(question, choice):

    question_emb = embedding_from_string(question)
    choice_emb = embedding_from_string(choice)

    similarity_score = cosine_similarity(question_emb, choice_emb)
    return similarity_score

def get_choices_sorted(question, choices):
    similarity_scores = [get_similarity(question, choice) for choice in choices]

    # Sort choices based on similarity scores and select top choices
    sorted_choices = [choice for _, choice in sorted(zip(similarity_scores, choices), reverse=True)]
    return sorted_choices


def select_best_n_choices(data,wsd_dict,n=4):

    formatted_data=[]
    sum =0

    #ddict = {}
    # Iterate over objects in the first file
    for obj1 in data:
        lemma_id_to_search = obj1['lemma_id']

        # Find all objects with the same lemma_id in the dictionary
        matching_objects = [obj2 for obj2 in wsd_dict if obj2.get('lemma_id') == lemma_id_to_search]

        obj1['context'] = preprocess_function(obj1['context'])
        for e in matching_objects:
            e['gloss'] = preprocess_function(e['gloss'])

        #use camel-bert to sort the glosses according to their similarity to the context
        sorted_choices = get_choices_sorted(obj1['context'], [obj2['gloss'] for obj2 in matching_objects])

        # Define a custom sorting key function
        def custom_sort_key(item):
            return sorted_choices.index(item["gloss"])

        # Sort the JSON list using the custom sorting key
        sorted_matched_objects = sorted(matching_objects, key=custom_sort_key)


        formatted_entry = {}

        formatted_entry['label'] =-1

        selected_choices = sorted_matched_objects[:n]
        for i,choice in enumerate(selected_choices):
            field_name = f"gloss{i}"
            formatted_entry[field_name] = choice['gloss']
            if choice['gloss_id'] == obj1['gloss_id']:
                formatted_entry['label'] = i

        #if correct choice is not covered skip the sample
        if formatted_entry['label'] == -1:
            print(obj1)
            print("***************************************\n")
            sum+=1
            continue


        #complete the number of choices to the max choice num (n)
        for i in range(len(selected_choices),n,1):
            field_name = f"gloss{i}"
            formatted_entry[field_name] = None


        formatted_entry["context_id"]= obj1['context_id']
        formatted_entry["context"]= obj1['context']
        formatted_entry["word"]=  obj1['word']
        formatted_entry["gloss_id"] = obj1['gloss_id']
        formatted_entry["lemma_id"] = obj1['lemma_id']

        formatted_data.append(formatted_entry)

        # Deletes the JSON object with index true_gloss from the list and returns it
        #removed_object = matching_objects.pop(true_gloss)

        #print(removed_object)

        #for i,x in enumerate(sorted_choices):
            #if removed_object['gloss'] == x:
                #if str(i) not in ddict:
                    #ddict[str(i)] = 1
                #else:
                    #ddict[str(i)] +=1

                #if i > 4:
                    #sum+=1
            #print(x+"\n")
    print(f"Number of ignored instances: {sum}\n")
    return formatted_data

num_choices = 4
formatted_data = select_best_n_choices(dev_set,wsd_dict,num_choices)

with open("wsd/camel_dev4_new.json","w") as file:
  json.dump(formatted_data,file,indent=4, ensure_ascii=False)

{'context_id': 'context.321', 'context': 'والجراثيم المسببة للأمراض مختلفة باختلاف المرض', 'word': 'للأمراض', 'lemma_id': 'ar.320', 'gloss_id': 'gloss.340'}
***************************************

{'context_id': 'context.330', 'context': 'محتويات  تطعيمات الأطفال  فوائد تطعيم الأطفال  الأمراض التي تغطى بالتطعيمات  أنواع تطعيمات الأطفال  كيفية تطعيم الأطفال  الآثار الجانبية لتطعيمات الأطفال  جدول التطعيمات في بعض الدول العربية  فيديو تعريف المطاعيم وكيفية التعامل معها  المراجع ما هي تطعيمات الأطفال تطعيمات الأطفال تعد المطاعيم التي تعطى للأطفال وسيلة لخلق مناعة تجاه بعض الأمراض التي من الممكن أن تصيبهم', 'word': 'الأمراض', 'lemma_id': 'ar.320', 'gloss_id': 'gloss.340'}
***************************************

{'context_id': 'context.371', 'context': 'ويأتي دور المطعوم بتحفيز الجهاز المناعي لخلق رد فعل مشابه لما قد يحدث في حالة الإصابة بالعدوى', 'word': 'حالة', 'lemma_id': 'ar.357', 'gloss_id': 'gloss.515'}
***************************************

{'context_id': 'context.523', 'context'

In [ ]:
import unicode_charset as ucc
import re

noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
def punctuations_remove(text):

    ar_str = u''.join(ucc.UNICODE_PUNCT_SYMBOL_CHARSET)

    clean_text = re.sub(r'['+re.escape(ar_str)+ r']+', '', text)
    return clean_text

def diacritics_remove(text):
    text = re.sub(noise, '', text)
    return text

def number_remove(text):
    text = re.sub("\d+", "", text)
    return text

def remove_english(text):
    text = re.sub(r'[a-zA-Z]', '', text)
    return text.strip()


max_input_length = 1024
max_target_length = 10

def preprocess_function(gloss):
    gloss = punctuations_remove(gloss)
    gloss = diacritics_remove(gloss)
    gloss = number_remove(gloss)
    gloss = gloss.strip()
    return gloss


In [ ]:

def load_data(file):
    # Specify the path to your JSON file
    json_file_path = f"{file}.json"

    try:
        with open(json_file_path, 'r', encoding='utf-8') as file:
            data_str = file.read()
            # Decode Unicode escape sequences
            data_str_decoded = data_str.encode()
            data = json.loads(data_str_decoded)

            # Check if the JSON file is not empty
            if data:
                # Print the first object
                #print(json.dumps(data[0], indent=4, ensure_ascii=False))
                first_object = data[0]
                attributes = first_object.keys()
                print(attributes)
            else:
                print("JSON file is empty.")
        return data
    except FileNotFoundError:
        print(f"File '{json_file_path}' not found.")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")

In [ ]:
#load_dictionary
wsd_dict = load_data("/content/drive/MyDrive/WSD task/WSD_dict")
print(len(wsd_dict))

dict_keys(['lemma_id', 'gloss_id', 'gloss'])
15865


In [ ]:
#load train data
train_set= load_data("train")
print(len(train_set))

dict_keys(['context_id', 'context', 'word', 'lemma_id', 'gloss_id'])
22404


In [ ]:
train_set[0]

{'context_id': 'context.300',
 'context': 'تحد تقني عالمي،',
 'word': 'تحد',
 'lemma_id': 'ar.300',
 'gloss_id': 'gloss.200'}

In [ ]:
#load train data
dev_set= load_data("/content/drive/MyDrive/WSD task/dev")
print(len(dev_set))

dict_keys(['context_id', 'context', 'word', 'lemma_id', 'gloss_id'])
2801


In [ ]:
#create training set for Binary Classification
import sys
import random

def formate_data_for_bin_classification(data,wsd_dict):
    formatted_data=[]

    # Iterate over objects in the first file
    for obj1 in data:
        lemma_id_to_search = obj1['lemma_id']

        # Find all objects with the same lemma_id in the dictionary
        matching_objects = [obj2 for obj2 in wsd_dict if obj2.get('lemma_id') == lemma_id_to_search]


        # Iterate over the list and add each element as a new field to the JSON object
        for i, value in enumerate(matching_objects):
            formatted_entry = {}
            formatted_entry["context_id"]= obj1['context_id']
            formatted_entry["context"]= obj1['context']
            formatted_entry["word"]=  obj1['word']
            formatted_entry['sense'] = value['gloss']
            formatted_entry['lemma_id']= value['lemma_id']
            formatted_entry['gloss_id'] = value['gloss_id']
            if value['gloss_id'] == obj1['gloss_id']:
                formatted_entry['label'] = 1
            else:
                formatted_entry['label'] = 0

            formatted_data.append(formatted_entry)

    return formatted_data


formatted_data = formate_data_for_bin_classification(dev_set,wsd_dict)

with open("wsd/bin_dev.json","w") as file:
  json.dump(formatted_data,file,indent=4, ensure_ascii=False)

In [ ]:
#find the maximum possible number of choices for set of data
def get_statistics(data,wsd_dict):

    stats = {}
    lemma_freq = {}

    max_num_choices = -1
    # Iterate over objects in the first file
    for obj1 in data:
        lemma_id_to_search = obj1['lemma_id']

        if lemma_id_to_search in lemma_freq:
            continue

        # Find all objects with the same lemma_id in the dictionary
        matching_objects = [obj2 for obj2 in wsd_dict if obj2.get('lemma_id') == lemma_id_to_search]

        lemma_freq[lemma_id_to_search] = len(matching_objects)

        if str(len(matching_objects)) not in stats.keys():
            stats[str(len(matching_objects))] = set()

        stats[str(len(matching_objects))].add(lemma_id_to_search)

        if len(matching_objects)>max_num_choices:
            max_num_choices=len(matching_objects)

    return max_num_choices, lemma_freq, stats

max_num_choices, lemma_freq, stats = get_statistics(train_set,wsd_dict)
print(max_num_choices)
print(len(lemma_freq))
print(len(stats))


# Sorting the dictionary based on keys
#sorted_dict_lambda = dict(sorted(lemma_freq.items(), key=lambda item: item[0]))

#for k,v in sorted_lemma_freq.items():
#    print(f"{k}:{v}\n")

#max_num_choices = find_max_numb_choices(dev_set,wsd_dict)
#print(max_num_choices)

43
4242
36


In [ ]:
kk=str()
ss =0
st=0
print("Number of choices\t Number of lemmas with that number \n")
for k,v in stats.items():
    if int(k) <=15:
        kk += f"{k}:{len(v)}\n"
        ss += len(v)
    else:
        print(f"{k}:{len(v)}\n")
        st+=len(v)
print(ss)
print(st)
print(kk)


Number of choices	 Number of lemmas with that number 

16:6

21:3

20:6

32:2

30:1

27:2

24:2

19:4

17:11

18:5

31:2

36:1

26:3

23:1

41:1

39:1

25:2

43:1

34:1

28:2

22:1

4184
58
5:286
11:32
9:65
8:82
4:486
1:1098
2:1009
6:185
15:15
3:706
10:42
7:124
13:15
14:20
12:19



In [ ]:
#create training set for MCT
import sys
import random

max_num_choices = 4

def formate_data_for_MCT(data,wsd_dict):
    formatted_data=[]

    # Iterate over objects in the first file
    for obj1 in data:
        lemma_id_to_search = obj1['lemma_id']

        # Find all objects with the same lemma_id in the dictionary
        matching_objects = [obj2 for obj2 in wsd_dict if obj2.get('lemma_id') == lemma_id_to_search]

        print(obj1)
        print(matching_objects)
        sys.exit()

        true_gloss = -1

        for i,x in enumerate(matching_objects):
            if x['gloss_id']== obj1['gloss_id']:
                true_gloss = i
                break

        # Deletes the JSON object with index true_gloss from the list and returns it
        removed_object = matching_objects.pop(true_gloss)

        #select 3 false and 1 correct glosses
        matching_objects = [removed_object] + matching_objects[:max_num_choices]

        # Shuffle the list in place
        random.shuffle(matching_objects)


        formatted_entry = {}
        # Iterate over the list and add each element as a new field to the JSON object
        for i, value in enumerate(matching_objects):
            field_name = f"gloss{i}"
            formatted_entry[field_name] = value['gloss']
            if value['gloss_id'] == obj1['gloss_id']:
                formatted_entry['label'] = i

        #complete the number of choices to the max possible number
        for i in range(len(matching_objects),max_num_choices,1):
            field_name = f"gloss{i}"
            formatted_entry[field_name] = None


        formatted_entry["context_id"]= obj1['context_id']
        formatted_entry["context"]= obj1['context']
        formatted_entry["word"]=  obj1['word']


        formatted_data.append(formatted_entry)
        # Print the updated JSON object
        #print(json.dumps(formatted_entry, indent=4,ensure_ascii=False))
        #sys.exit()

    return formatted_data


formatted_data = formate_data_for_MCT(train_set,wsd_dict)

with open("wsd/proc_train15.json","w") as file:
  json.dump(formatted_data,file,indent=4, ensure_ascii=False)


{'context_id': 'context.300', 'context': 'تحد تقني عالمي،', 'word': 'تحد', 'lemma_id': 'ar.300', 'gloss_id': 'gloss.200'}
[{'lemma_id': 'ar.300', 'gloss_id': 'gloss.200', 'gloss': 'ما يواجَه من عقبات أو أخطار'}, {'lemma_id': 'ar.300', 'gloss_id': 'gloss.201', 'gloss': 'مباراة رياضيّة تتيح للفائز الاحتفاظ بشيء إلى أن ينتزعه منه فائز جديد يتغلّب عليه'}, {'lemma_id': 'ar.300', 'gloss_id': 'gloss.202', 'gloss': 'نظريّة في فلسفة التاريخ مؤدّاها: أنّ الحضارة تنشأ عندما يواجه شعب ما تحدِّيًا يهدِّد كيانه فيواجه هذا التحدِّي ببذل جهد مضاعف استجابة لحبِّ البقاء.'}, {'lemma_id': 'ar.300', 'gloss_id': 'gloss.203', 'gloss': 'مشكلة أو أُحجيّة محفِّزة للعقل.'}, {'lemma_id': 'ar.300', 'gloss_id': 'gloss.204', 'gloss': 'امتحان لقدرات المرء وطاقاتِه'}]


SystemExit: 

/media/nvme1n1/yyounes/anaconda3/envs/htc/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#here we select the best matching 4 options

#create training set for MCT
import sys
import random

from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

#given a dictionary of questions and its answers. sort the answers according to their cosine similarity to the questions
def sort_lemmas_according_to_similarity(question_choices):

    # Define TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    # Dictionary to store reduced choices
    sorted_choices = defaultdict(list)

    # Iterate over questions and choices
    for question, choices in questions_choices.items():
        # Transform question and choices into TF-IDF vectors
        documents = [question] + choices
        tfidf_matrix = vectorizer.fit_transform(documents)

        # Compute cosine similarity between question and choices
        similarity_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])[0]

        # Sort choices based on similarity scores and select top choices
        sorted_choices = [choice for _, choice in sorted(zip(similarity_scores, choices), reverse=True)]
        reduced_choices[question] = sorted_choices  #[:3] # Adjust the number of choices to keep

    return sorted_choices

def get_choices_sorted(question, choices):
     # Transform question and choices into TF-IDF vectors
    documents = [question] + choices
    tfidf_matrix = vectorizer.fit_transform(documents)

    # Compute cosine similarity between question and choices
    similarity_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])[0]

    # Sort choices based on similarity scores and select top choices
    sorted_choices = [choice for _, choice in sorted(zip(similarity_scores, choices), reverse=True)]

    return sorted_choices


max_num_choices = 4


def select_best_n_choices(data,wsd_dict,n=4):

    formatted_data=[]
    sum =0

    # Iterate over objects in the first file
    for obj1 in data:
        lemma_id_to_search = obj1['lemma_id']

        # Find all objects with the same lemma_id in the dictionary
        matching_objects = [obj2 for obj2 in wsd_dict if obj2.get('lemma_id') == lemma_id_to_search]

        sorted_choices = get_choices_sorted(obj1['context'], [obj2['gloss'] for obj2 in matching_objects])


        #print(obj1)
        true_gloss = -1

        for i,x in enumerate(matching_objects):
            if x['gloss_id']== obj1['gloss_id']:
                true_gloss = i
                break

        # Deletes the JSON object with index true_gloss from the list and returns it
        removed_object = matching_objects.pop(true_gloss)

        #print(removed_object)

        for i,x in enumerate(sorted_choices):
            if removed_object['gloss'] == x:
                if i > 3:
                    #print(i)
                    sum+=1
            #print(x+"\n")
    print(sum)


formatted_data = select_best_n_choices(train_set,wsd_dict,4)


'''
        #select 3 false and 1 correct glosses
        matching_objects = [removed_object] + matching_objects[:max_num_choices]

        # Shuffle the list in place
        random.shuffle(matching_objects)


        formatted_entry = {}
        # Iterate over the list and add each element as a new field to the JSON object
        for i, value in enumerate(matching_objects):
            field_name = f"gloss{i}"
            formatted_entry[field_name] = value['gloss']
            if value['gloss_id'] == obj1['gloss_id']:
                formatted_entry['label'] = i

        #complete the number of choices to the max possible number
        for i in range(len(matching_objects),max_num_choices,1):
            field_name = f"gloss{i}"
            formatted_entry[field_name] = None


        formatted_entry["context_id"]= obj1['context_id']
        formatted_entry["context"]= obj1['context']
        formatted_entry["word"]=  obj1['word']


        formatted_data.append(formatted_entry)
        # Print the updated JSON object
        #print(json.dumps(formatted_entry, indent=4,ensure_ascii=False))
        #sys.exit()

    return formatted_data


formatted_data = select_best_n_choices(train_set,wsd_dict,4)

with open("wsd/proc_train15.json","w") as file:
  json.dump(formatted_data,file,indent=4, ensure_ascii=False)
'''

8
6
7
4
6
5
5
15
4
4
5
5
5
5
11
4
5
5
8
4
12
6
5
30
6
6
15
7
6
19
6
6
4
6
5
5
6
8
4
13
6
4
9
4
8
4
8
4
4
5
18
5
5
4
4
5
4
5
4
6
5
4
5
5
8
12
10
4
10
4
4
5
6
4
6
5
8
4
5
5
9
4
5
6
8
5
6
6
6
4
6
6
5
5
6
5
8
10
8
4
16
6
5
9
8
23
6
5
6
6
5
10
6
4
4
5
4
12
4
4
18
6
8
7
7
4
10
12
4
6
6
4
4
11
6
13
7
4
4
7
10
4
6
15
4
7
6
6
6
7
6
7
7
4
5
5
11
7
11
5
11
6
6
10
6
6
8
6
20
6
8
4
30
4
4
4
7
4
4
8
6
4
8
4
5
4
4
4
10
5
6
8
6
6
5
5
5
4
6
4
5
4
4
5
5
10
5
6
7
7
7
5
7
4
4
20
4
4
7
4
4
5
5
4
4
4
6
4
4
6
29
6
9
4
9
4
4
4
6
5
12
12
4
5
5
4
6
5
4
5
8
4
8
4
4
4
4
13
4
6
4
5
7
5
5
22
7
5
6
5
5
5
4
4
4
8
8
4
11
7
16
7
4
4
9
14
8
9
5
6
12
9
5
8
9
10
9
5
4
8
8
14
11
9
5
9
6
4
6
6
6
8
6
7
10
4
10
4
8
5
4
5
4
7
5
7
5
5
7
5
7
5
8
5
8
7
4
10
5
6
8
7
6
4
11
6
7
5
7
6
6
4
6
7
20
10
7
6
4
9
9
4
7
5
4
9
5
7
9
9
6
4
9
5
10
4
6
8
4
6
7
11
4
4
8
6
8
9
5
6
6
4
6
6
12
4
7
5
4
14
5
6
9
5
5
5
4
4
6
5
4
6
4
10
6
4
7
4
5
7
5
5
6
9
5
10
4
4
4
4
5
5
8
5
9
4
11
5
9
6
5
4
4
4
8
4
6
10
6
8
6
5
4
6
6
5
5
6
15
6
6
9
4
9
6
9
4
4
4
4
5

'\n        #select 3 false and 1 correct glosses\n        matching_objects = [removed_object] + matching_objects[:max_num_choices]\n\n        # Shuffle the list in place\n        random.shuffle(matching_objects)\n    \n      \n        formatted_entry = {}\n        # Iterate over the list and add each element as a new field to the JSON object\n        for i, value in enumerate(matching_objects):\n            field_name = f"gloss{i}"\n            formatted_entry[field_name] = value[\'gloss\']\n            if value[\'gloss_id\'] == obj1[\'gloss_id\']:\n                formatted_entry[\'label\'] = i\n\n        #complete the number of choices to the max possible number\n        for i in range(len(matching_objects),max_num_choices,1):\n            field_name = f"gloss{i}"\n            formatted_entry[field_name] = None\n            \n                \n        formatted_entry["context_id"]= obj1[\'context_id\']\n        formatted_entry["context"]= obj1[\'context\']\n        formatted_entry["w

In [ ]:
#using SimcSE with ARABERTV02
#filtering using camelbert

from transformers import BertTokenizer, BertForSequenceClassification
import torch

model_checkpoint = "../SimCSE/result/wsd_camel_bert"  #"CAMeL-Lab/bert-base-arabic-camelbert-msa"


#trained_camel_bert = "CAM/bert-base-arabic-camelbert-msa_best_model"

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)#CAMeL-Lab/bert-base-arabic-camelbert-msa") #"bert-base-multilingual-uncased") #bert-base-arabic-camelbert-ca
model = BertForSequenceClassification.from_pretrained(model_checkpoint)#CAMeL-Lab/bert-base-arabic-camelbert-msa")#bert-base-multilingual-uncased")


# Define function to calculate contextual similarity
def get_similarity(question, choice):
    inputs = tokenizer(question, choice, return_tensors="pt", max_length=512, truncation=True, padding=True)
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    similarity_score = probabilities[0][1].item()  # Assuming binary classification
    return similarity_score

def get_choices_sorted(question, choices):
    similarity_scores = [get_similarity(question, choice) for choice in choices]

    # Sort choices based on similarity scores and select top choices
    sorted_choices = [choice for _, choice in sorted(zip(similarity_scores, choices), reverse=True)]
    return sorted_choices


def select_best_n_choices(data,wsd_dict,n=4):

    formatted_data=[]
    sum =0

    #ddict = {}
    # Iterate over objects in the first file
    for obj1 in data:
        lemma_id_to_search = obj1['lemma_id']


        # Find all objects with the same lemma_id in the dictionary
        matching_objects = [obj2 for obj2 in wsd_dict if obj2.get('lemma_id') == lemma_id_to_search]

        #use camel-bert to sort the glosses according to their similarity to the context
        sorted_choices = get_choices_sorted(obj1['context'], [obj2['gloss'] for obj2 in matching_objects])

        # Define a custom sorting key function
        def custom_sort_key(item):
            return sorted_choices.index(item["gloss"])

        # Sort the JSON list using the custom sorting key
        sorted_matched_objects = sorted(matching_objects, key=custom_sort_key)


        formatted_entry = {}

        formatted_entry['label'] =-1

        selected_choices = sorted_matched_objects[:n]
        for i,choice in enumerate(selected_choices):
            field_name = f"gloss{i}"
            formatted_entry[field_name] = choice['gloss']
            if choice['gloss_id'] == obj1['gloss_id']:
                formatted_entry['label'] = i

        #if correct choice is not covered skip the sample
        if formatted_entry['label'] == -1:
            print(obj1)
            print("***************************************\n")
            sum+=1
            continue


        #complete the number of choices to the max choice num (n)
        for i in range(len(selected_choices),n,1):
            field_name = f"gloss{i}"
            formatted_entry[field_name] = None


        formatted_entry["context_id"]= obj1['context_id']
        formatted_entry["context"]= obj1['context']
        formatted_entry["word"]=  obj1['word']
        formatted_entry["gloss_id"] = obj1['gloss_id']
        formatted_entry["lemma_id"] = obj1['lemma_id']

        formatted_data.append(formatted_entry)

        # Deletes the JSON object with index true_gloss from the list and returns it
        #removed_object = matching_objects.pop(true_gloss)

        #print(removed_object)

        #for i,x in enumerate(sorted_choices):
            #if removed_object['gloss'] == x:
                #if str(i) not in ddict:
                    #ddict[str(i)] = 1
                #else:
                    #ddict[str(i)] +=1

                #if i > 4:
                    #sum+=1
            #print(x+"\n")
    print(f"Number of ignored instances: {sum}\n")
    return formatted_data

num_choices = 4
formatted_data = select_best_n_choices(dev_set,wsd_dict,num_choices)

with open("wsd/simcse_arabertv2_dev4.json","w") as file:
  json.dump(formatted_data,file,indent=4, ensure_ascii=False)

/media/nvme1n1/yyounes/anaconda3/envs/htc/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../SimCSE/result/wsd_camel_bert and are newly initialized: ['classifier.bias', 'bert.pooler.dense.bias', 'classifier.weight', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'context_id': 'context.353', 'context': 'محتويات ١ تطعيمات الأطفال ٢ فوائد تطعيم الأطفال ٣ الأمراض التي تُغطى بالتطعيمات ٤ أنواع تطعيمات الأطفال ٥ كيفية تطعيم الأطفال ٦ الآثار الجانبية لتطعيمات الأطفال ٧ جدول التطعيمات في بعض الدول العربية ٨ فيديو تعريف المطاعيم وكيفية التعامل معها ٩ المراجع ما هي تطعيمات الأطفال تطعيمات الأطفال تُعد المطاعيم التي تُعطى للأطفال وسيلة لخلق مناعة تجاه بعض الأمراض التي من الممكن أن تصيبهم،', 'word': 'تصيبهم،', 'lemma_id': 'ar.343', 'gloss_id': 'gloss.430'}
***************************************

{'context_id': 'context.366', 'context': 'ويأتي دور المطعوم بتحفيز الجهاز المناعي لخلق رد فعل مشابه لما قد يحدث في حالة الإصابة بالعدوى،', 'word': 'الجهاز', 'lemma_id': 'ar.353', 'gloss_id': 'gloss.472'}
***************************************

{'context_id': 'context.465', 'context': '[٤] القضاء على العديد من الأمراض التي كانت سببََا لإصابة أو وفاة الآلاف من الأطفال،', 'word': 'سببََا', 'lemma_id': 'ar.417', 'gloss_id': 'gloss.764'}
****************************

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'context_id': 'context.4658', 'context': 'ومنها تظهر قائمة لمربع حوار ندخل متغير المنتج ومتغير الاصناف بالشكل', 'word': 'تظهر', 'lemma_id': 'ar.472', 'gloss_id': 'gloss.1078'}
***************************************

{'context_id': 'context.4681', 'context': 'والرسم اليدوي يكون بجعل المحور السيني المحافظات والصادي الانتاج وبالاعتماد على قيم الانتاج يحدد ارتفاع كل شريط .', 'word': 'يحدد', 'lemma_id': 'ar.790', 'gloss_id': 'gloss.2601'}
***************************************

{'context_id': 'context.4683', 'context': '1-6-2 الدائرة البيانية (pie chart) :', 'word': 'الدائرة', 'lemma_id': 'ar.1240', 'gloss_id': 'gloss.4607'}
***************************************

{'context_id': 'context.4694', 'context': '- عبارة عن دائرة مقسمة الى مقاطع والتي تصف مكونات نسب المجتمع او العينة التي تعود الى اصناف مختلفة ،', 'word': 'اصناف', 'lemma_id': 'ar.1323', 'gloss_id': 'gloss.4954'}
***************************************

{'context_id': 'context.4818', 'context': '5,20) ويظهر موقعها في الشكل البي

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

{'context_id': 'context.13443', 'context': 'لما تمارسه الأخيرة من تدخلات خطيرة ومرفوضة في الشؤون الداخلية للمملكة المغربية،', 'word': 'الشؤون', 'lemma_id': 'ar.1102', 'gloss_id': 'gloss.3968'}
***************************************

{'context_id': 'context.13529', 'context': 'أو عدمِ توحيدِ شكلِ المُنتَج بِشكلٍ عامٍ, وكانت كُلُّ الأسباب السَّابقة تُؤدِّي،', 'word': 'الأسباب', 'lemma_id': 'ar.417', 'gloss_id': 'gloss.764'}
***************************************

{'context_id': 'context.13720', 'context': 'تاريـخ 2020/12/1 م ،', 'word': 'تاريـخ', 'lemma_id': 'ar.402', 'gloss_id': 'gloss.698'}
***************************************

{'context_id': 'context.13743', 'context': 'كتاب التمارين ) الصف الرابع ( / المركز الوطني لتطوير المناهج .', 'word': 'المركز', 'lemma_id': 'ar.588', 'gloss_id': 'gloss.1648'}
***************************************

{'context_id': 'context.13778', 'context': 'حركةُ الأرض.', 'word': 'الأرض.', 'lemma_id': 'ar.705', 'gloss_id': 'gloss.2189'}
******************

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'context_id': 'context.22814', 'context': 'جملة تصف كيفية ارتباط عناصر المجال بعناصر المدى.', 'word': 'المدى.', 'lemma_id': 'ar.812', 'gloss_id': 'gloss.2709'}
***************************************

{'context_id': 'context.23067', 'context': 'ممــنّْ لــم يكونــوا مهتميــن باكتشــاف أراٍضٍ جديـدة بقـدر مـلء سـفنهم بــ “أصنـاف” جديـدة.', 'word': '“أصنـاف”', 'lemma_id': 'ar.1323', 'gloss_id': 'gloss.4954'}
***************************************

{'context_id': 'context.23123', 'context': 'وقال بيان وزارة الدفاع الهولندية في هذا الشأن إن وقوع انفجارات ثانوية غير متوقعة يعني احتمال ارتفاع عدد القتلى إلى مستوى يفوق التوقعات ما ادى لسقوط مدنيين علاوة على مقاتلين في التنظيم.', 'word': 'الشأن', 'lemma_id': 'ar.1102', 'gloss_id': 'gloss.3968'}
***************************************

{'context_id': 'context.23221', 'context': 'قال الفريق بالقوات الجوية الأمريكية جون هيسترمان في مؤتمر صحفي إن "قذائف صغيرة جدا أُلقيت على منشأة معروفة لتجهيز السيارات المفخخة الكائن بمنطقة صناعية.', 'word': 'أُ

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'context_id': 'context.26459', 'context': 'من جهتهم أكد أعضاء المحكمة الدستورية عزمهم على بذل أقصى الجهود في أداء عملهم من أجل تعزيز قوة القانون والقضاء.', 'word': 'عملهم', 'lemma_id': 'ar.710', 'gloss_id': 'gloss.2226'}
***************************************

{'context_id': 'context.26505', 'context': 'ولم يكن ليحدث هذا التطور لولا استخدام علم الديناميك الحراري في صناعة المحركات إذ تعتبر المحركات الحرارية من التطبيقات العملية لعلم الديناميك الحراري ،', 'word': 'صناعة', 'lemma_id': 'ar.1159', 'gloss_id': 'gloss.4237'}
***************************************

{'context_id': 'context.26513', 'context': 'إذ يتم التركيز على تحويل الطاقة الحرارية إلى طاقة ميكانيكية.', 'word': 'تحويل', 'lemma_id': 'ar.1588', 'gloss_id': 'gloss.5910'}
***************************************

{'context_id': 'context.26537', 'context': 'ويعتبر عام 1768م نقطة تحول في تاريخ البشرية ،', 'word': 'تاريخ', 'lemma_id': 'ar.402', 'gloss_id': 'gloss.697'}
***************************************

{'context_id': 'contex

In [ ]:

for k,v in formatted_data.items():
    print(f"{k}:{v}")


0:20125
1:1782
2:318
3:102
5:14
8:2
4:44
14:1
6:12
7:2
9:4


In [ ]:
num_choices = 5
formatted_data = select_best_n_choices(dev_set,wsd_dict,num_choices)

with open("wsd/camel_dev5.json","w") as file:
  json.dump(formatted_data,file,indent=4, ensure_ascii=False)

{'context_id': 'context.849', 'context': 'رد فعل تحسّسي شديد،', 'word': 'رد', 'lemma_id': 'ar.354', 'gloss_id': 'gloss.481'}
***************************************

{'context_id': 'context.1221', 'context': 'يا ليتني فيها جذع أكون حيا حين يخرجك قومك .', 'word': 'يخرجك', 'lemma_id': 'ar.651', 'gloss_id': 'gloss.1944'}
***************************************

{'context_id': 'context.1916', 'context': 'نرى أنّ المحقّق يزداد مهارةً في التعرُّف على تلك الأعمال الفنية التي هي نسخ طبق الأصل.', 'word': 'الأصل.', 'lemma_id': 'ar.767', 'gloss_id': 'gloss.2508'}
***************************************

{'context_id': 'context.2522', 'context': 'و ما آلت إليه حالهم بدعوته المباركة ،', 'word': 'حالهم', 'lemma_id': 'ar.357', 'gloss_id': 'gloss.524'}
***************************************

{'context_id': 'context.2742', 'context': 'والبيانات بخصوص الثروة المادية للبلاد تعطي فكرةً حول المصادر الماليةِ وحجمها ِ.', 'word': 'المصادر', 'lemma_id': 'ar.1271', 'gloss_id': 'gloss.4790'}
*******************

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'context_id': 'context.6565', 'context': 'ويحدثُ غالبًا في الكلاب وخاصةً البطباط وأكيتا،', 'word': 'وخاصةً', 'lemma_id': 'ar.756', 'gloss_id': 'gloss.2445'}
***************************************



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'context_id': 'context.7511', 'context': 'تقع بين خطي عرض 30 درجة و 38 درجة شمالاً وخطي طول 7 درجات و 12 درجة شرقاً.', 'word': 'خطي', 'lemma_id': 'ar.986', 'gloss_id': 'gloss.3464'}
***************************************

{'context_id': 'context.9208', 'context': 'وربط جديس بثمود التي كانت موجودة في الألف الأول قبل الميلاد', 'word': 'الأول', 'lemma_id': 'ar.577', 'gloss_id': 'gloss.1591'}
***************************************

{'context_id': 'context.9884', 'context': 'و ذكرت ذات المتحدثة, أن المداخلات صبت في تفكيك طبيعة العلاقات بين اللغة العربية والعلوم من خلال اعمال تطبيقية عبرت عن انفتاح افاقها المعرفية والفكرية من خلال تعالقات اللغوي بالرياضي والفلكي والموسيقي والاقتصادي وغيرها من الحقول المعرفية.', 'word': 'اللغوي', 'lemma_id': 'ar.2515', 'gloss_id': 'gloss.9286'}
***************************************

{'context_id': 'context.11564', 'context': 'أمسـك فجـأة بيـد سـولانج وقربهـا مـن يـده الأخـرى،', 'word': 'وقربهـا', 'lemma_id': 'ar.1278', 'gloss_id': 'gloss.4818'}
*********

...

{'context_id': 'context.14015', 'context': 'وحتى المجتمعات غير المسلمة تهتم بها الوزارة وتحل نزاعاتها لان لها قيم ترفع هذه النزاعات والمشكلات لذلك نحن كمسلمين نتعامل مع غير المسلمين بمشتركات الإنسانية ونتعامل من منطلق ديننا الحنيف ونتعامل مع جميع المخالفين فضلا ان يكون مسلم في طائفة من الطوائف.', 'word': 'ترفع', 'lemma_id': 'ar.2273', 'gloss_id': 'gloss.8443'}
***************************************

{'context_id': 'context.14404', 'context': 'فيقومون بربط أجسادهم بالأسلاك الحادّة،', 'word': 'الحادّة،', 'lemma_id': 'ar.3319', 'gloss_id': 'gloss.11634'}
***************************************

{'context_id': 'context.16776', 'context': 'ﻧﺠﺢ رواد ﻓﻀﺎء ﻣﻜﻮك اﻟﻔﻀﺎء اﻣﻴﺮﻛﻲ (أتلانتس) ﻓﻲ ﺗﺮﻛﻴﺐ زوج ﺟﺪﻳﺪ ﻣﻦ اﻟﻮاح اﻟﺸﻤﺴﻴﺔ ﺑﺎﻟﻤﺤﻄﺔ اﻟﻔﻀﺎﺋﻴﺔ اﻟﺪوﻟﻴﺔ ﻣﺎ ﺟﻌﻠﻬﺎ ﺗﺒﺪو أﻣﺲ الاول ذات أﺟﻨﺤﺔ ﺟﺪﻳﺪة ﺗﻤﺪﻫﺎ ﺑﻘﺪرات إﺿﺎﻓﻴﺔ ﻟﺘﻮﻟﻴﺪ اﻟﻄﺎﻗﺔ.', 'word': 'ﺗﻤﺪﻫﺎ', 'lemma_id': 'ar.1208', 'gloss_id': 'gloss.4452'}
***************************************

{'context_id': 'context.17102', 'context': 'ويستجلب خيار افعالهم ويتج

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'context_id': 'context.25650', 'context': 'تمت الترجمة والتحرير اللغوي والجمع التصويري في شعبة اللغة العربية إدارة الخدمات المؤسسية والمنشآت،', 'word': 'والتحرير', 'lemma_id': 'ar.2276', 'gloss_id': 'gloss.8473'}
***************************************



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'context_id': 'context.27762', 'context': 'إذ إن ثمة أنظمة تبنى لمعالجة أغراض لا تتجاوز الحاجة فيها إلا لتعيين الاسم والفعل والحرف ،', 'word': 'والحرف', 'lemma_id': 'ar.1481', 'gloss_id': 'gloss.5514'}
***************************************

Number of ignored instances: 27



In [ ]:
import json

# Read the JSON file
with open('wsd/camel_train5.json', 'r') as file:
    data = json.load(file)

# Iterate through each item in the JSON data
for item in data:
    # Check if the "Label" field exists in the current item
    if 'Label' in item:
        # Rename the field from "Label" to "label"
        item['label'] = item.pop('Label')

# Write the modified JSON back to the file
with open('wsd/camel_train5555.json', 'w') as file:
    json.dump(data, file, indent=4)


In [ ]:
For WSD, first we fine-tune Camel-BERT on the dataset for 10 epochs using 32 as batch size an. It achieves 94.75% accuracy. Then we use it to select the most four relevant choices. After that, we use mBERT in
Multiple choice task. Where we consider the context as question and the most relevant choices obtained by Camel-BERT as the the possible answers.